In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models import *

In [2]:
train_tape = "Z:/rppg/ccnu_datatape_300x5x5_train.h5"
valid_tape = "Z:/rppg/ccnu_datatape_300x5x5_valid.h5"

train = load_datatape(train_tape)
valid = load_datatape(valid_tape)

def to_rhythm(datatape, dtype=tf.float16):
    s = datatape.shape
    def _():
        for i, j in datatape:
            i = i.reshape((s[0], s[1]*s[2], s[3]))
            i = cv2.cvtColor(i.astype(np.float32), cv2.COLOR_RGB2YUV)
            j = [get_hr(j)]
            yield i, j
    return tf.data.Dataset.from_generator(lambda :_(), output_types=(dtype, dtype), output_shapes=((s[0], s[1]*s[2], s[3]), (1,)))

train, valid = to_rhythm(train).cache(), to_rhythm(valid).cache()

In [3]:
"""
To use GRU and the smoothing loss mentioned in the original text, you should turn off heart rate data augmentation.
i.e., set extend_rate=0 in dump_datatape.
"""

class RhythmNet(keras.Model):

    def __init__(self, use_gru=True):
        super().__init__()
        self.ResNet = ResNet([2, 2, 2, 2])

        # GRU layer
        if use_gru:
            self.ResNet.fc = keras.Sequential([layers.Reshape((1, -1)) ,layers.GRU(1, return_sequences=True), layers.Reshape((-1,))])

    def call(self, x):
        x = self.ResNet(x)
        return tf.reshape(x, (-1,))
    
def rhythm_loss(label, pred, r=100):
    return tf.abs(label-pred) + r*tf.reduce_mean(tf.abs(tf.reduce_mean(label)-pred))

In [4]:
rhythmnet = RhythmNet(use_gru=False)
rhythmnet.build(input_shape=(None, 300, 25, 3))
rhythmnet.compile(optimizer='adam', loss='mae')
rhythmnet.summary()

Model: "rhythm_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
res_net (ResNet)             multiple                  11188865  
Total params: 11,188,865
Trainable params: 11,181,057
Non-trainable params: 7,808
_________________________________________________________________


In [5]:
rhythmnet.fit(train.shuffle(2000).batch(32), validation_data=valid.batch(32), epochs=50, callbacks=[keras.callbacks.ModelCheckpoint('../weights/RhythmNet_CCNU.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
rhythmnet.load_weights('../weights/RhythmNet_CCNU.h5')

In [6]:
def rhythmnet_e2e(x):
    s = x.shape
    x = x.reshape((s[0], s[1], s[2]*s[3], s[4]))
    x = np.array([cv2.cvtColor(i.astype(np.float32), cv2.COLOR_RGB2YUV) for i in x])
    return rhythmnet(x)
    

In [16]:
eval_on_dataset(test_set_CCNU, rhythmnet_e2e, 300, (5, 5), output='HR', step=3, batch=32, save='../results/RhythmNet_CCNU_CCNU.h5')
get_metrics('../results/RhythmNet_CCNU_CCNU.h5')

100%|██████████| 179/179 [00:13<00:00, 12.87it/s]


{'Sliding window': {'MAE': 10.303, 'RMSE': 13.746, 'R': -0.11835},
 'Whole video': {'MAE': 9.729, 'RMSE': 13.24, 'R': -0.1354}}

In [15]:
eval_on_dataset(test_set_CCNU_rPPG, rhythmnet_e2e, 300, (5, 5), output='HR', step=3, batch=32, save='../results/RhythmNet_CCNU_CCNU_rPPG.h5')
get_metrics('../results/RhythmNet_CCNU_CCNU_rPPG.h5')

100%|██████████| 54/54 [00:03<00:00, 17.62it/s]


{'Sliding window': {'MAE': 10.431, 'RMSE': 14.595, 'R': -0.12622},
 'Whole video': {'MAE': 9.975, 'RMSE': 14.203, 'R': -0.12317}}

In [14]:
eval_on_dataset(test_set_UBFC_rPPG2, rhythmnet_e2e, 300, (5, 5), output='HR', step=3, batch=32, save='../results/RhythmNet_CCNU_UBFC.h5')
get_metrics('../results/RhythmNet_CCNU_UBFC.h5')

100%|██████████| 42/42 [00:02<00:00, 20.10it/s]


{'Sliding window': {'MAE': 24.416, 'RMSE': 29.298, 'R': -0.19648},
 'Whole video': {'MAE': 24.086, 'RMSE': 29.147, 'R': -0.22832}}

In [13]:
eval_on_dataset(test_set_PURE, rhythmnet_e2e, 300, (5, 5), output='HR', step=3, batch=32, save='../results/RhythmNet_CCNU_PURE.h5')
get_metrics('../results/RhythmNet_CCNU_PURE.h5')

100%|██████████| 59/59 [00:02<00:00, 21.83it/s]


{'Sliding window': {'MAE': 24.682, 'RMSE': 27.567, 'R': 0.17528},
 'Whole video': {'MAE': 24.734, 'RMSE': 27.612, 'R': 0.13707}}